#Tropical Rainfall Analysis

This program will use python packages to create a series of classes that can be used to create a configurable tropical rainfall analysis dashboard. The present algorithm gives the swath of TMPA 3B42 along the track of hurricane Florence. The input files are florence.txt which has been create dfrom hurdat_extended.txt file.


In [2]:
# installing required libraries
!apt-get update
!apt-get install -qq  libgdal-dev libproj-dev 
!pip uninstall -y shapely
!pip install shapely cartopy --no-binary shapely --no-binary cartopy
!pip install pydap


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:12 http://archive.ubuntu.com/ubuntu

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
# import libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import numpy as np
import datetime as dt
from types import SimpleNamespace

# creating dates function in the same format as in "florence.txt" file
def dates(input):
    if len(str(input[1]))==5:
        datefor='0'+str(input[1])
    year=str(input[0])
    month=datefor[0:2]
    date=datefor[2:4]
    hour=datefor[4:6]
    dm = dt.datetime.strptime(year+'.'+month+'.'+date+'.'+hour, '%Y.%m.%d.%H')
    return dm.timetuple(),datefor


track_file='/content/drive/My Drive/TC_Contribution/florence.txt'              # location of the florence.txt file.

data=pd.read_fwf(track_file, sep=" ", header=None)
track_lon=-data[5].values
track_lat=data[4].values

#=================================================================
# Setting the track plot
proj = ccrs.Mercator()
ll_proj = ccrs.Geodetic()
bound_lons = [-105, -6.5]
bound_lats = [0, 40]
fig, ax = plt.subplots(subplot_kw=dict(projection=proj))
ax.set_extent(bound_lons+bound_lats, ll_proj)
land = ax.add_feature(cfeature.LAND, facecolor='burlywood')
coast = ax.add_feature(cfeature.COASTLINE)
ocean = ax.add_feature(cfeature.OCEAN, facecolor='steelblue')
country = ax.add_feature(cfeature.BORDERS, edgecolor='gray')
gl=ax.gridlines(draw_labels=True, linestyle=':')
plt.plot(track_lon,track_lat,color='blue', linewidth=2,
         transform=ccrs.Geodetic())
plt.close()
#plt.show()
#=================================================================

# creating function to extract TMPA3B42RT data. 
def download_trmm(mdate,md):
	from pydap.client import open_url
	from pydap.cas.urs import setup_session
	trmm=SimpleNamespace()
	yyyymmddHH=str(mdate.tm_year)+md[0:2]+md[2:4]+md[4:6]
	julian=mdate.tm_yday-1 if md[4:6]=='00' else mdate.tm_yday                   # setting the julian date
	opendap_url="https://disc2.gesdisc.eosdis.nasa.gov/opendap/TRMM_RT/TRMM_3B42RT.7/2018/"+str(julian)+"/3B42RT."+yyyymmddHH+".7.nc4"
	
	# login for earthdata at https://urs.earthdata.nasa.gov/
	username = 'tiwari13'
	password = '########'
	session = setup_session(username, password, check_url=opendap_url)
	dataset = open_url(opendap_url, session=session)
	
	#slicing the lat-longs
	trmm.lat=dataset['lat'].data[slice(250,None)]                              
	trmm.lon=dataset['lon'].data[slice(None,750)]

  # creating precipitation files
	pcp=dataset['precipitation'].data[slice(250,None),slice(None,750)]
	pcp[pcp<0]=np.nan
	trmm.pcp=pcp
	return trmm

# creating function to track the precipitation from TRMM for the lat-longs in the florence text file.
def track_data(trmm,tc_cen_lon,tc_cen_lat,boundary_extent=6):
	lon_id=np.where(np.logical_and(trmm.lon>tc_cen_lon-boundary_extent, trmm.lon<tc_cen_lon+boundary_extent))[0]
	lat_id=np.where(np.logical_and(trmm.lat>tc_cen_lat-boundary_extent, trmm.lat<tc_cen_lat+boundary_extent))[0]
	pcp_crop=trmm.pcp[lat_id[0]:lat_id[-1]+1,lon_id[0]:lon_id[-1]+1]
	pcp_nan=np.empty((trmm.pcp.shape[0],trmm.pcp.shape[1]))*np.nan
	pcp_nan[lat_id[0]:lat_id[-1]+1,lon_id[0]:lon_id[-1]+1]=pcp_crop
	trmm.pcpcrop=pcp_nan
	return trmm

# creating function to do countor plot using matplot and cartopy to draw globe cropping the desired lat-long.
def mycontourf(trmm,varname):
	import matplotlib.pyplot as plt
	import cartopy.crs as ccrs
	from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
	import matplotlib.ticker as mticker
	fig=plt.figure(figsize=(15,7))
	ax = plt.axes(projection=ccrs.PlateCarree())
	ax.set_extent([-180,180,-60,60])

	ax.coastlines(resolution="110m",linewidth=1)
	gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='black', linestyle='--')
	gl.xlabels_top = False
	gl.ylabels_right = False
	gl.xlines = True
#	gl.xlocator = mticker.FixedLocator([-180, -90, 0, 90, 180])
#	gl.ylocator = mticker.FixedLocator([-60, -50, -25, 0, 25, 50, 60])
	gl.xformatter = LONGITUDE_FORMATTER
	gl.yformatter = LATITUDE_FORMATTER
	gl.xlabel_style = {'size':16, 'color':'black'}
	gl.ylabel_style = {'size':16, 'color':'black'}

	plt.contourf(trmm.lon,trmm.lat,varname)
	plt.axis('equal')
	plt.colorbar()
	plt.show()

pdata=[]

# i is the time period, ranging from 1 to 77 with a time step of 2, it can be varied depending on how descriptive swath we want.
for i in range(1,77,2):
	print(i)
	start=np.array([data[3][i],data[2][i]])
	mdate,md=dates(start)
	trmm=download_trmm(mdate,md)
	out=track_data(trmm,track_lon[i],track_lat[i],boundary_extent=10)
	print(out.pcpcrop.shape)
	pdata.append(out.pcpcrop)

outputdata=np.dstack(pdata)
outputdata[outputdata<0.5]=np.nan
mycontourf(trmm,np.nanmean(outputdata,axis=2))


1


ConnectionError: ignored